<a href="https://colab.research.google.com/github/kiiwee/opengen/blob/master/Phi-3/Phi_3_Medium_4K_Instruct_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="align-center">
  <a href="https://www.fontys.nl/"><img src="https://www.fontys.nl/static/design/FA845701-BD71-466E-9B3D-38580DFAD5B4-fsm/images/logo-inverted@2x.png" width="120"></a>
  <!-- <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://www.arise-biodiversity.nl/"><img src="https://i.imgur.com/fwGepON.png" width="200"></a>
  <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://diopsis.eu/"><img src="https://i.imgur.com/e6nutxp.png" width="180"></a> -->
</div>

# **In this notebook we will train a Phi-3 Medium 4K Instruct model on a custom dataset using the Unsloth framework**

---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/kiiwee/opengen)

<a href="https://github.com/facebookresearch/detectron2"><img src="https://raw.githubusercontent.com/unslothai/unsloth/main/images/unsloth%20logo%20white%20text.png" width="80"></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# First we install the unsloth library:

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

# And load the base model to be fine-tuned:

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # RoPE Scaling internally
dtype = None # Auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-medium-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# We now add LoRA adapters so we only need to update 1 to 10% of all parameters:

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 0 is optimized
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = "unsloth", # "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.6 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


# Next we load the train and validation dataset from HuggingFace:

In [4]:
from datasets import load_dataset

train_dataset = load_dataset("kalinkov/tailwindcss_components", split = "train")
eval_dataset = load_dataset("kalinkov/tailwindcss_components", split = "validation")

# And we setup the trainer parameters:

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 0,
        num_train_epochs=1,
        # eval params
        fp16_full_eval = True,
        per_device_eval_batch_size = 2,
        eval_accumulation_steps = 4,
        evaluation_strategy = "steps",
        eval_steps = 10,
        save_strategy = "steps",
        save_steps = 10,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/content/drive/MyDrive/data_challenge/Evaluation_training",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
7.504 GB of memory reserved.


# We initiate the initial 1 epoch training:

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,683 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 210
 "-____-"     Number of trainable parameters = 65,536,000


Step,Training Loss
1,0.757500
2,0.897400
3,0.810100
4,0.891500
5,0.869300
6,0.805800
7,0.809700
8,0.803600
9,0.693200
10,0.671100


Step,Training Loss
1,0.757500
2,0.897400
3,0.810100
4,0.891500
5,0.869300
6,0.805800
7,0.809700
8,0.803600
9,0.693200
10,0.671100


In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,683 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 210
 "-____-"     Number of trainable parameters = 65,536,000


Step,Training Loss,Validation Loss
10,0.686100,0.730386
20,0.516000,0.647077
30,0.553700,0.609974
40,0.547600,0.587576
50,0.462400,0.569451
60,0.522200,0.553456
70,0.542800,0.538971
80,0.593900,0.527049
90,0.492900,0.519172
100,0.424100,0.514378


# Followed by 2 more epochs:

In [11]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 0,
        num_train_epochs=3,
        # eval params
        fp16_full_eval = True,
        per_device_eval_batch_size = 2,
        eval_accumulation_steps = 4,
        evaluation_strategy = "steps",
        eval_steps = 10,
        save_strategy = "steps",
        save_steps = 10,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/content/drive/MyDrive/data_challenge/Evaluation_training",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer_stats = trainer.train(resume_from_checkpoint = True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,683 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 630
 "-____-"     Number of trainable parameters = 65,536,000


Step,Training Loss,Validation Loss
430,0.343300,0.451122
440,0.258500,0.455547
450,0.304900,0.452578
460,0.252300,0.454047
470,0.390100,0.451694
480,0.255100,0.449968
490,0.319600,0.446997
500,0.382600,0.445371
510,0.268500,0.445946
520,0.291500,0.447992


# We look at the training results throughout the fine-tuning process:

In [29]:
import pandas as pd
loss_history = pd.DataFrame(trainer.state.log_history)

In [45]:
loss_history[loss_history["step"] % 10 == 0][30:50]

,epoch,grad_norm,learning_rate,loss,step,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
174,0.760095,0.171091,0.000049,0.5050,160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,0.760095,NaN,NaN,NaN,160,0.484153,62.6687,3.191,1.596,NaN,NaN,NaN,NaN,NaN
185,0.807601,0.159732,0.000039,0.4307,170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,0.807601,NaN,NaN,NaN,170,0.482501,62.4476,3.203,1.601,NaN,NaN,NaN,NaN,NaN
196,0.855107,0.199525,0.000029,0.3930,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,0.855107,NaN,NaN,NaN,180,0.481835,62.6386,3.193,1.596,NaN,NaN,NaN,NaN,NaN
207,0.902613,0.150071,0.000020,0.3564,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,0.902613,NaN,NaN,NaN,190,0.479189,62.6743,3.191,1.596,NaN,NaN,NaN,NaN,NaN
218,0.950119,0.174251,0.000010,0.4638,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,0.950119,NaN,NaN,NaN,200,0.477668,62.7070,3.189,1.595,NaN,NaN,NaN,NaN,NaN


In [43]:
loss_history.to_csv('/content/drive/MyDrive/data_challenge/Evaluation_training/loss_history.csv')

# Getting the desired checkpoint:

In [ ]:
trainer.train(resume_from_checkpoint='/content/drive/MyDrive/data_challenge/Evaluation_training/checkpoint-210/')

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

15551.9819 seconds used for training.
259.2 minutes used for training.
Peak reserved memory = 9.92 GB.
Peak reserved memory for training = 2.416 GB.
Peak reserved memory % of max memory = 67.263 %.
Peak reserved memory for training % of max memory = 16.382 %.


# Saving the finetuned model

In [ ]:
# model.save_pretrained("/content/drive/MyDrive/data_challenge/LoRa/lora_model") # Local saving
model.push_to_hub("kalinkov/phi3_lora_model", token = "hf_...") # Online saving

Saved model to https://huggingface.co/kalinkov/phi3_lora_model


### GGUF / llama.cpp Conversion

Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.

In [ ]:
# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if True: model.push_to_hub_gguf("kalinkov/Phi3_tailwindcss", tokenizer, quantization_method = "q4_k_m", token = "hf_...")

Unsloth: Will remove a cached repo with size 1.2K


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 34.36 out of 50.99 RAM for saving.


100%|██████████| 40/40 [01:15<00:00,  1.88s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Extending kalinkov/Phi3_tailwindcss/tokenizer.model with added_tokens.json.
Originally tokenizer.model is of size (32000).
But we need to extend to sentencepiece vocab size (32011).


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at kalinkov/Phi3_tailwindcss into f16 GGUF format.
The output location will be ./kalinkov/Phi3_tailwindcss/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: Phi3_tailwindcss
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 4096
INFO:hf-to-gguf:gguf: embedding length = 5120
INFO:hf-to-gguf:gguf: feed forward length = 17920
INFO:hf-to-gguf:gguf: head count = 40
INFO:hf-to-gguf:gguf: key-value head count = 10
INFO:hf-to-gguf:gguf: rope theta

unsloth.F16.gguf:   0%|          | 0.00/27.9G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/kalinkov/Phi3_tailwindcss
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q4_K_M.gguf:   0%|          | 0.00/8.40G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/kalinkov/Phi3_tailwindcss
